# Initialization

Change to `valuenet` directory and add `src` path to `PYTHONPATH`.

In [ ]:
%cd /home/ec2-user/SageMaker/valunet_clean

In [2]:
import sys
sys.path.insert(0, '/home/ec2-user/SageMaker/valunet_clean/src')

# Train Model

Here also, we follow the [user manual from Valuenet](https://github.com/brunnurs/valuenet).

In [ ]:
# Empty gpu use, might be useful to 'clean' your memory
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
!nvidia-smi
# kill -9 PID

## Training data

The data that will be used for training is located in the files
- valuenet/data/**dataset**/train.json
- valuenet/data/**dataset**/dev.json

with **dataset** being either hack_zurich or spider.

### Option A: Train with Spider dataset

In [3]:
data_set = 'spider'

### Option B: Train with Custom Data only

Create your custom data and preprocess it following the steps of [preprocess_custom_data-01.ipynb](https://github.com/hack-with-admin-ch/aws-sagemaker-notebook-valuenet/blob/main/preprocess_custom_data-01.ipynb). 

Then, set dataset to 'hack_zurich' (as the cell below). 

In [ ]:
data_set = 'hack_zurich'

### Option C: Train with Spider Dataset and the Custom Data

In this case, merge your custom data and spider data together in one json file for train, one for dev and one for the tables:
Concatenate
- valuenet/data/hack_zurich/train.json with valuenet/data/spider/train.json
- valuenet/data/hack_zurich/dev.json with valuenet/data/spider/dev.json
- valuenet/data/hack_zurich/original/tables.json with valuenet/data/spider/original/tables.json

and save the merged files in hack_zurich valuenet/data/hack_zurich/train.json, valuenet/data/hack_zurich/dev.json and valuenet/data/hack_zurich/original/tables.json.

As fine-tuning was never experimented with the Valuenet codebase, it is the safest way to train a model which will perform better on custom data.

In [ ]:
data_set = 'hack_zurich'

## Training procedure

**Hyperparameters**:
Here, we set a few parameters but do not hesitate to look at [training parameters](https://github.com/brunnurs/valuenet/blob/hackzurich/src/config.py) if you want to specify other settings.

Train from scratch (with exception of the transformer encoder, which is already pre-trained). Fine-tuning was never implemented with valuent, feel free to try !

In [2]:
exp_name = "train-01" # Name of the training (saved in wandb and folder name in experiments)
batch_size = 8 # make it smaller if you encounter memory limitations 
num_epochs = 40

In [ ]:
%run src/main.py --exp_name {exp_name} --data_set {data_set} --cuda --batch_size {batch_size} --num_epochs {num_epochs}

**Results and final model**:
After training, you will be able to find the ground truth data, output of the model, evaluation results and intermediary data in the folder `valuenet/experiments/{exp_name}_date_time`.
If the accuracy was better than the last best accuracy, the model is also saved in the folder as `"best_model.pt"`.